In [48]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
pd.set_option('display.float_format', lambda x: '%.1f' % x)
# gcloud auth application-default login

In [49]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical_new_int", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [50]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-10-09 20:47:43,0,None,nan,nan,325,iPhone_Ordering,RT @TattedUpBreezy: I really hate when people ...,917491788188999685,...,469,23121,15241,2739284219,Nation Wide,iPhones For Sale🌟,64384,Pacific Time (US & Canada),0,1
1,1,2017-10-09 20:47:42,2,None,nan,nan,0,AzuReverie,iPhone 8+ portrait mode was a cool idea until ...,917491787522105346,...,408,1313,955,16290629,"Houston, TX, USA",AzuP☆☆★,248271,Central Time (US & Canada),0,0
2,2,2017-10-09 20:47:42,0,None,nan,nan,10,deena_mahmod,RT @Solitairian: For sale :\nNew iPhone 7 plus...,917491787153002496,...,4976,1335,654,747628098141720576,,ديـنا,13411,None,0,1
3,3,2017-10-09 20:47:42,2,None,nan,nan,0,gayvapecat,The iPhone 8 has a better battery life than I do,917491786985234434,...,2187,104,166,625000210,Rev Squad❤️,kyle😤,3904,Hawaii,0,0
4,4,2017-10-09 20:47:41,0,None,nan,nan,244,HWhistler,RT @CommonWhiteGirI: Petition for nintedogs fo...,917491782266556416,...,7905,470,473,579288535,north cackalacky,🐡hęather ❂,21464,None,0,1


In [51]:
df.describe()

,index,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,18356.0,18356.0,1626.0,1956.0,18356.0,18356.0,18356.0,18356.0,18356.0,18356.0,18356.0,18356.0,18356.0
mean,49.2,1.0,915790412222072320.0,115583838490984512.0,1799.1,916171831977768576.0,7610.1,14871.2,2320.8,181212606435458144.0,59786.0,0.0,0.4
std,28.8,29.4,4828063232734422.0,285653485510214080.0,9983.1,1252165020060385.2,21188.4,202636.5,10378.1,342276263890144128.0,157880.6,0.1,0.5
min,0.0,0.0,794138753157758976.0,586.0,0.0,914292134601093120.0,0.0,0.0,0.0,1942.0,0.0,0.0,0.0
25%,24.0,0.0,914967926152831232.0,70824567.0,0.0,914976961305419776.0,74.0,153.0,149.0,259004515.2,2492.8,0.0,0.0
50%,49.0,0.0,915001825872459520.0,481629472.0,0.0,915001004703293440.0,1065.0,536.0,431.0,1573032520.0,12642.0,0.0,0.0
75%,74.0,0.0,917462675296155520.0,3032313791.0,114.0,917471466923473920.0,6128.8,2071.0,1339.0,4103949820.5,49673.0,0.0,1.0
max,99.0,3011.0,917490846509003008.0,917086485303758976.0,143000.0,917491788188999680.0,858992.0,18833639.0,440682.0,917489926765023232.0,7933108.0,1.0,1.0


In [52]:
df.count()

index                      18356
created_at                 18356
favorite_count             18356
in_reply_to_screen_name     1956
in_reply_to_status_id       1626
in_reply_to_user_id         1956
retweet_count              18356
screen_name                18356
text                       18356
tweet_id                   18356
tweet_location                38
user_created_at            18356
user_description           18356
user_favorites_count       18356
user_followers_count       18356
user_friends_count         18356
user_id                    18356
user_location              18356
user_name                  18356
user_statuses_count        18356
user_time_zone             11387
user_verified              18356
RT                         18356
dtype: int64

In [53]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [54]:
dfs.describe()

,index,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,18342.0,18342.0,1626.0,1956.0,18342.0,18342.0,18342.0,18342.0,18342.0,18342.0,18342.0,18342.0,18342.0
mean,49.2,1.0,915790412222072320.0,115583838490984512.0,1800.5,916172746277046144.0,7609.9,14881.5,2321.8,181230687425078624.0,59761.1,0.0,0.4
std,28.8,29.4,4828063232734422.0,285653485510214080.0,9986.8,1252205201586566.5,21187.6,202713.5,10381.9,342304647313119360.0,157869.0,0.1,0.5
min,0.0,0.0,794138753157758976.0,586.0,0.0,914292134601093120.0,0.0,0.0,0.0,1942.0,0.0,0.0,0.0
25%,24.0,0.0,914967926152831232.0,70824567.0,0.0,914976997135311872.0,74.0,153.0,149.2,259013293.0,2503.0,0.0,0.0
50%,49.0,0.0,915001825872459520.0,481629472.0,0.0,915001012924049408.0,1065.0,536.0,431.0,1573887733.0,12648.0,0.0,0.0
75%,74.0,0.0,917462675296155520.0,3032313791.0,114.0,917471482161250304.0,6130.2,2071.0,1339.0,4105181763.5,49657.2,0.0,1.0
max,99.0,3011.0,917490846509003008.0,917086485303758976.0,143000.0,917491788188999680.0,858992.0,18833639.0,440682.0,917489926765023232.0,7933108.0,1.0,1.0


In [55]:
dfs.count()

index                      18342
created_at                 18342
favorite_count             18342
in_reply_to_screen_name     1956
in_reply_to_status_id       1626
in_reply_to_user_id         1956
retweet_count              18342
screen_name                18342
text                       18342
tweet_id                   18342
tweet_location                38
user_created_at            18342
user_description           18342
user_favorites_count       18342
user_followers_count       18342
user_friends_count         18342
user_id                    18342
user_location              18342
user_name                  18342
user_statuses_count        18342
user_time_zone             11378
user_verified              18342
RT                         18342
dtype: int64

In [56]:
cols = list(dfs)
#cols - https://stackoverflow.com/a/25122293

Delete duplicates which have all same tweet text

In [57]:
dfd = dfs.drop_duplicates(subset = 'text', inplace=False)

In [58]:
dfd.describe()

,index,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,12672.0,12672.0,1625.0,1950.0,12672.0,12672.0,12672.0,12672.0,12672.0,12672.0,12672.0,12672.0,12672.0
mean,49.0,1.5,915790917147826688.0,115471061064975616.0,273.6,916119005578348672.0,6611.2,18726.1,2477.1,165871046803545024.0,69073.0,0.0,0.2
std,28.7,35.4,4829506532077617.0,285461431040680096.0,3713.9,1248840618115838.0,19205.8,231424.6,11529.4,331248585641719744.0,179547.5,0.2,0.4
min,0.0,0.0,794138753157758976.0,586.0,0.0,914292134601093120.0,0.0,0.0,0.0,13328.0,1.0,0.0,0.0
25%,24.0,0.0,914967894064321024.0,72312019.5,0.0,914977709800005632.0,35.0,155.0,129.0,198436399.0,2639.0,0.0,0.0
50%,49.0,0.0,915001846116228992.0,484110255.0,0.0,914999742964320256.0,672.5,593.0,420.0,1190414004.0,14376.5,0.0,0.0
75%,74.0,0.0,917462684383064064.0,3033427655.0,0.0,917471745682126848.0,4611.2,2395.2,1363.0,3382066771.8,58269.0,0.0,0.0
max,99.0,3011.0,917490846509003008.0,917086485303758976.0,143000.0,917491788188999680.0,486443.0,18833639.0,440682.0,917489926765023232.0,7933108.0,1.0,1.0


In [59]:
# drop Nan if it is in specific column
ddp = dfd.dropna(axis=0, subset=['in_reply_to_screen_name'], how='any')

In [60]:
ddp.describe()

,index,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,1950.0,1950.0,1625.0,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0,1950.0
mean,48.5,0.7,915790917147826432.0,115471061064975616.0,0.1,916238955034277504.0,8424.0,37673.0,843.1,164283869029019520.0,53551.1,0.1,0.0
std,28.3,4.7,4829506532077617.0,285461431040680096.0,0.7,1248893636528700.8,21146.0,159738.1,3393.4,331479588707558976.0,154838.5,0.3,0.0
min,0.0,0.0,794138753157758976.0,586.0,0.0,914540655443156992.0,0.0,0.0,0.0,601963.0,1.0,0.0,0.0
25%,24.0,0.0,914967894064321024.0,72312019.5,0.0,914978466209033216.0,193.0,91.0,101.2,160198330.8,1051.5,0.0,0.0
50%,49.0,0.0,915001846116228992.0,484110255.0,0.0,917455626569121792.0,1566.5,335.5,296.0,1048666881.5,6149.5,0.0,0.0
75%,72.0,1.0,917462684383064064.0,3033427655.0,0.0,917473376833181696.0,7118.2,1423.5,690.0,3337918699.5,27057.0,0.0,0.0
max,99.0,157.0,917490846509003008.0,917086485303758976.0,29.0,917491775727702016.0,474618.0,1389271.0,67713.0,917487580660293632.0,1738055.0,1.0,1.0


In [61]:
ddp.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
6,6,2017-10-09 20:47:40,0,bebfoodman,917489298722696960.0,2200868494.0,0,SachinNakrani,@bebfoodman ha! It's an iPhone ... and a lions...,917491775727702016,...,2317,17887,814,1560323995,London,Sachin Nakrani,84683,London,1,0
48,48,2017-10-09 20:46:52,0,SamsungSupport,nan,18768513.0,0,AnneBarschall,@SamsungSupport I downloaded your printer app ...,917491576254775296,...,25,208,385,460289535,"Tarrytown, NY, USA",AnneBarschall,4860,Central Time (US & Canada),0,0
50,50,2017-10-09 20:46:48,0,LosPollosTV,917490784613442048.0,2464343990.0,0,FerociousDRG,@LosPollosTV Iphone sucks get a Pixel 2,917491559582588930,...,109,9,84,901515501851095040,GB,Ferocious,122,Pacific Time (US & Canada),0,0
63,63,2017-10-09 20:46:38,0,AskRBC,nan,65709615.0,0,TravisTurnerTO,@AskRBC the iPhone app on my 8plus doesn’t hav...,917491516335108097,...,15956,430,358,52776097,"Toronto, Ontario, Canada",Travis Turner,618,Eastern Time (US & Canada),0,0
68,68,2017-10-09 20:46:32,2,LosPollosTV,917490784613442048.0,2464343990.0,0,BestHD123,@LosPollosTV You should do a donation goal for...,917491491781693441,...,1077,15,72,739479411662159872,United States,BestHD123,903,Central Time (US & Canada),0,0


In [62]:
ddp.to_csv("twitter_dataset_v3.csv")